<a href="https://colab.research.google.com/github/hlshao/1132_class/blob/main/sampleCode.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 加上自己學號: 12345

# Set up the Notebook

In [1]:
#import pandas and numpy
import pandas as pd
import numpy as np

#PCA
from sklearn.decomposition import PCA

#import standard classification tools from sklearn
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.ensemble import RandomForestClassifier
import itertools

#classification model
from sklearn.model_selection import train_test_split

#visualization
import matplotlib.pyplot as plt
import seaborn as sns

#plotly
import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff

template = 'ggplot2'

KeyboardInterrupt: 

# Import and Prepare Data

In [ ]:
#load data
data = pd.read_table('https://raw.githubusercontent.com/PineBiotech/omicslogic/master/CellLines_52samples_ExprData_T1.txt',sep='\t',header=(0))
features = data.iloc[1:, 0].values

#add the gene id to index column
data.index = data.id

#drop or remove "id" column from the data
data=data.drop(['id'], axis = 1)

#transpose
dataT = np.transpose(data)

#prepare train and test data
X = dataT.iloc[:, 1:].values
y = dataT.iloc[:, 0].values

#save class names
classes = np.unique(y)
n_classes = len(classes)
cat = dataT['class']

#prepare train and test data (try changing test_size, enable/disable stratification)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.4, random_state=0, stratify = y)

#print out test class proportion from total
for i in range(len(classes)):
  print(classes[i], y_test.tolist().count(classes[i]), "/", y.tolist().count(classes[i]))

In [ ]:
dataT = dataT.drop(columns=['class'])
dataT.head()

In [ ]:
#Run Principal Component Analysis
pca = PCA(n_components=3)

pca.fit(dataT)
xpca = pca.transform(dataT)

PC1_label = "PC1 {}%".format(round((100*pca.explained_variance_ratio_[0]),2))
PC2_label = "PC2 {}%".format(round((100*pca.explained_variance_ratio_[1]),2))
PC3_label = "PC3 {}%".format(round((100*pca.explained_variance_ratio_[2]),2))

#create dataframe for PCA
df_pca = pd.DataFrame(xpca)
df_pca.columns = [PC1_label,PC2_label,PC3_label]

#visualize
figPCA = px.scatter_3d(df_pca, x=PC1_label, y=PC2_label, z=PC3_label,
                       title="PCA 3D plot, colored by Class", color=cat,
                       template="plotly_white", height=800, width=800)
figPCA.show()

In [ ]:
from IPython.display import Image

img_bytes = figPCA.to_image(format="png", width=800, height=800, scale=1)
Image(img_bytes)

# Random Forest

In [ ]:
#define Random forest model
model = RandomForestClassifier(n_estimators = 100,
                               max_depth = 500,
                               verbose=False,
                               random_state=0)

#fit model on training data
model.fit(X_train, y_train)

#Predict test data
predictions = model.predict(X_test)
print("Model accuracy = ", accuracy_score(y_test, predictions))

In [ ]:
#define Random forest model
model = RandomForestClassifier(n_estimators = 1000,
                               max_depth = 5000,
                               bootstrap=True,
                               verbose=False,
                               random_state=0)

#fit model on training data
model.fit(X_train, y_train)

#Predict test data
predictions = model.predict(X_test)
print("Model accuracy = ", accuracy_score(y_test, predictions))

## Confusion Matrix

In [ ]:
#prepare a confusion matrix
conf = confusion_matrix(y_test,predictions)
new_conf = pd.DataFrame(conf, columns=classes, index=classes)

#plot heatmap of confusion matrix
fig, ax = plt.subplots(figsize=(13, 10))
sns.heatmap(new_conf, annot=True);

##Feature Importance

In [ ]:
#prepare the list of significant features
importances = model.feature_importances_
std = np.std([tree.feature_importances_ for tree in model.estimators_], axis=0)

#select top features and save them
forest_importances = pd.Series(importances, index=features)
forest_importances = forest_importances[forest_importances > 0.0025]
forest_importances = forest_importances.sort_values()

#plot forest_importances
forest_importances.plot.bar(figsize=(15, 7), title="top features by RF gini");

#print
print("total number of selected features: ", len(forest_importances))

# PCA


In [ ]:
selected_features = forest_importances.index.tolist()

dataT1 = dataT[dataT.columns.intersection(selected_features)]
print("new dimensions of select features: ", dataT1.shape)

#Run Principal Component Analysis
pca = PCA(n_components=3)
pca.fit(dataT1)
xpca = pca.transform(dataT1)

PC1_label = "PC1 {}%".format(round((100*pca.explained_variance_ratio_[0]),2))
PC2_label = "PC2 {}%".format(round((100*pca.explained_variance_ratio_[1]),2))
PC3_label = "PC3 {}%".format(round((100*pca.explained_variance_ratio_[2]),2))

#create dataframe for PCA
df_pca = pd.DataFrame(xpca)
df_pca.columns = [PC1_label,PC2_label,PC3_label]

#visualize
figPCA = px.scatter_3d(df_pca, x=PC1_label, y=PC2_label, z=PC3_label,
                       title="PCA 3D plot, colored by Class", color=cat,
                       template="plotly_white", height=600, width=800)
figPCA.show()

In [ ]:
img_bytes = figPCA.to_image(format="png", width=800, height=800, scale=1)
Image(img_bytes)

# Questions for Reflection:
How did hyperparameters change accuracy of the model?
How many significant genes did we use for PCA?
How did the 'variance explained' of the PCA improve after feature selection?